# exploring 

In [12]:
import pandas as pd
import numpy as np
import datetime
import re
from scipy import stats

In [13]:
df = pd.read_csv("../Cleaned_Data/Tasks.csv")

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25012 entries, 0 to 25011
Data columns (total 11 columns):
Unnamed: 0          25012 non-null int64
User                25012 non-null object
Project             25012 non-null object
Task_Category       25012 non-null object
Task_Description    25012 non-null object
Date                25012 non-null object
Duration            25012 non-null object
Time_Decimal        25012 non-null float64
Cost_per_hour       25012 non-null float64
Position            25012 non-null object
Task_Cost           25012 non-null float64
dtypes: float64(3), int64(1), object(7)
memory usage: 2.1+ MB


In [15]:
# reording the data to make it more understandable
df.Cost_per_hour = df.Cost_per_hour.round(2)
df.Time_Decimal = df.Time_Decimal.round(2)
df.drop(columns=["Unnamed: 0"], inplace=True)

In [19]:
 df = df[["Date", "Duration", "Project", "Task_Category", "Task_Cost", "Task_Description", "User", "Position", "Cost_per_hour","Time_Decimal"]]

In [20]:
df

,Date,Duration,Project,Task_Category,Task_Cost,Task_Description,User,Position,Cost_per_hour,Time_Decimal
0,2017-09-01,01:17:00,Revenify,Development,26.41,Feedback on budgets index PR and Standup meeti...,UI13,Senior Back End Developer,20.58,1.28
1,2017-09-01,02:30:00,WWU,Development,62.50,Create graphs in XLSX format from user input,UI36,Senior Full-Stack Developer,25.00,2.50
2,2017-09-01,00:36:49,WWU,Development,15.34,Create graphs in XLSX format from user input,UI36,Senior Full-Stack Developer,25.00,0.61
3,2017-09-01,00:12:05,WWU,Development,5.03,Weekly Standup,UI38,Project Manager / Tech Lead,25.00,0.20
4,2017-09-01,00:09:07,WWU,Development,3.80,Weekly standup,UI36,Senior Full-Stack Developer,25.00,0.15
...,...,...,...,...,...,...,...,...,...,...
25007,2019-09-28,00:41:22,CareBetter,Development,24.13,[DB_ADMIN_002] Create user <> admin association,UI35,Senior Full-Stack Developer,35.00,0.69
25008,2019-09-28,00:20:55,CandidatePool,Development,8.46,Working on adding email and owner to applicant...,UI20,Back End Developer,24.26,0.35
25009,2019-09-28,01:02:00,CandidatePool,Development,29.02,[FrontEnd] : Add an Alert when The User does n...,UI14,Front End Developer,28.08,1.03
25010,2019-09-28,00:20:59,CareBetter,Development,12.24,[DB_ADMIN_002] Create user <> admin association,UI35,Senior Full-Stack Developer,35.00,0.35


In [28]:
df.columns

Index(['Date', 'Duration', 'Project', 'Task_Category', 'Task_Cost',
       'Task_Description', 'User', 'Position', 'Cost_per_hour',
       'Time_Decimal'],
      dtype='object')

In [53]:
# 24 projects! are massured
len(list(df.Project.unique()))

24

In [24]:
# turing the Duration into timedelata
df.Duration = pd.to_timedelta(df.Duration)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25012 entries, 0 to 25011
Data columns (total 10 columns):
Date                25012 non-null object
Duration            25012 non-null timedelta64[ns]
Project             25012 non-null object
Task_Category       25012 non-null object
Task_Cost           25012 non-null float64
Task_Description    25012 non-null object
User                25012 non-null object
Position            25012 non-null object
Cost_per_hour       25012 non-null float64
Time_Decimal        25012 non-null float64
dtypes: float64(3), object(6), timedelta64[ns](1)
memory usage: 1.9+ MB


# Avg cost:
1) by project
2) By catogory (Devlopment (Backend, Frontend, Full stack, Tech Lead) 
3) by seniorty 
4) calulate 


## Avg 
### all
median = 32.16 (from all projects)
mean = 32.13 (from all hours)
as we can see 32.1 USD is the avg of a programmer 
### over 1000 hours  

### under 1000 hours but more thne 100

median = 34.42 

In [44]:
# call the mean
df.Task_Cost.sum()/df.Time_Decimal.sum()

32.136385297567145

In [33]:
#cal the median per 
# to most expensive per hour project
exp_pro = df[['Project', 'Task_Category', 'Task_Cost', 'Duration', 'Time_Decimal']] 

In [41]:
cost_per_hour_per_pro = (exp_pro.groupby(by="Project").sum()
.assign(mean_cost_per_hour_per_pro= lambda x : round(x.Task_Cost / x.Time_Decimal, 2)))

In [42]:
cost_per_hour_per_pro.describe()


,Task_Cost,Time_Decimal,mean_cost_per_hour_per_pro
count,24.000000,24.000000,24.000000
mean,63923.702500,1989.137917,34.075833
std,95160.509228,3380.014861,10.289358
min,392.000000,13.970000,18.000000
25%,7706.195000,207.205000,26.637500
50%,22601.150000,710.495000,32.165000
75%,60723.685000,1766.385000,39.920000
max,365083.030000,14963.730000,64.600000


In [46]:
# lets take projects out are over 1000 hours  
len(cost_per_hour_per_pro[cost_per_hour_per_pro.Time_Decimal > 1000])
# it is 10 project!
# meadian is 34.42

10

In [69]:
= list(cost_per_hour_per_pro[cost_per_hour_per_pro.Time_Decimal > 1000].index)

['AgeSolvency',
 'CandidatePool',
 'Credious',
 'Finizon',
 'GDP',
 'NestPride',
 'PilotAxis',
 'Revenify',
 'TeachZip',
 'WeVape']

In [48]:
cost_per_hour_per_pro[cost_per_hour_per_pro.Time_Decimal > 1000].describe()

,Task_Cost,Time_Decimal,mean_cost_per_hour_per_pro
count,10.000000,10.000000,10.000000
mean,137911.711000,4325.511000,33.740000
std,112165.047036,4322.844214,7.031794
min,33014.990000,1178.090000,24.400000
25%,45966.075000,1488.270000,28.257500
50%,103270.190000,2670.550000,34.420000
75%,218455.507500,5271.385000,39.517500
max,365083.030000,14963.730000,42.040000


In [49]:
cost_per_hour_per_pro[cost_per_hour_per_pro.Time_Decimal > 1000]

,Task_Cost,Time_Decimal,mean_cost_per_hour_per_pro
Project,,,
AgeSolvency,163039.50,3878.65,42.04
CandidatePool,56206.17,1414.82,39.73
Credious,365083.03,14963.73,24.40
Finizon,238705.00,5735.63,41.62
GDP,74276.23,1939.68,38.29
NestPride,37047.82,1278.65,28.97
PilotAxis,132264.15,3401.42,38.88
Revenify,42552.71,1708.62,24.90
TeachZip,236927.51,7755.82,30.55


In [64]:
# lets take projects out are under 1000 hours  and over 100 hours
len(cost_per_hour_per_pro[(cost_per_hour_per_pro.Time_Decimal < 1000) & (cost_per_hour_per_pro.Time_Decimal > 100)])
# it is 11 projects

11

In [65]:
cost_per_hour_per_pro[(cost_per_hour_per_pro.Time_Decimal < 1000) & (cost_per_hour_per_pro.Time_Decimal > 100)].describe()

,Task_Cost,Time_Decimal,mean_cost_per_hour_per_pro
count,11.000000,11.000000,11.000000
mean,13728.880909,392.538182,36.827273
std,9169.777765,270.749075,12.835255
min,2536.420000,106.030000,18.000000
25%,7473.670000,198.460000,27.265000
50%,11944.460000,305.350000,37.630000
75%,17697.370000,525.440000,40.905000
max,33001.500000,893.570000,64.600000


# meeting from project


In [84]:
df[df.Task_Description.str.contains("[mM]eeting|[Ww]eekly|[Dd]aily|[Ss]tandup", regex=True)] 

,Date,Duration,Project,Task_Category,Task_Cost,Task_Description,User,Position,Cost_per_hour,Time_Decimal
0,2017-09-01,01:17:00,Revenify,Development,26.41,Feedback on budgets index PR and Standup meeti...,UI13,Senior Back End Developer,20.58,1.28
3,2017-09-01,00:12:05,WWU,Development,5.03,Weekly Standup,UI38,Project Manager / Tech Lead,25.00,0.20
4,2017-09-01,00:09:07,WWU,Development,3.80,Weekly standup,UI36,Senior Full-Stack Developer,25.00,0.15
5,2017-09-01,00:15:49,Credious,Development,6.59,Weekly Standup,UI38,Project Manager / Tech Lead,25.00,0.26
16,2017-09-04,00:44:10,Revenify,Development,15.15,Standup meeting,UI13,Senior Back End Developer,20.58,0.74
...,...,...,...,...,...,...,...,...,...,...
24983,2019-09-27,00:09:06,Credious,Development,4.26,XXX Daily Scrum,UI14,Front End Developer,28.08,0.15
24985,2019-09-27,00:11:25,CandidatePool,Development,4.11,Daily StandUp Meeting,UI16,Full-Stack Developer,21.61,0.19
24986,2019-09-27,00:10:00,CandidatePool,Development,4.04,Attending daily meeting,UI20,Back End Developer,24.26,0.17
24987,2019-09-27,00:10:49,CandidatePool,Development,5.06,XXX Daily Standup,UI14,Front End Developer,28.08,0.18


In [85]:
df[df.Task_Description.str.contains("[mM]eeting|[Ww]eekly|[Dd]aily|[Ss]tandup", regex=True) & (df.Time_Decimal < 1)]

,Date,Duration,Project,Task_Category,Task_Cost,Task_Description,User,Position,Cost_per_hour,Time_Decimal
3,2017-09-01,00:12:05,WWU,Development,5.03,Weekly Standup,UI38,Project Manager / Tech Lead,25.00,0.20
4,2017-09-01,00:09:07,WWU,Development,3.80,Weekly standup,UI36,Senior Full-Stack Developer,25.00,0.15
5,2017-09-01,00:15:49,Credious,Development,6.59,Weekly Standup,UI38,Project Manager / Tech Lead,25.00,0.26
16,2017-09-04,00:44:10,Revenify,Development,15.15,Standup meeting,UI13,Senior Back End Developer,20.58,0.74
20,2017-09-04,00:44:47,Credious,Development,18.66,Weekly Standup,UI38,Project Manager / Tech Lead,25.00,0.75
...,...,...,...,...,...,...,...,...,...,...
24983,2019-09-27,00:09:06,Credious,Development,4.26,XXX Daily Scrum,UI14,Front End Developer,28.08,0.15
24985,2019-09-27,00:11:25,CandidatePool,Development,4.11,Daily StandUp Meeting,UI16,Full-Stack Developer,21.61,0.19
24986,2019-09-27,00:10:00,CandidatePool,Development,4.04,Attending daily meeting,UI20,Back End Developer,24.26,0.17
24987,2019-09-27,00:10:49,CandidatePool,Development,5.06,XXX Daily Standup,UI14,Front End Developer,28.08,0.18


In [86]:
df[df.Task_Description.str.contains("[mM]eeting|[Ww]eekly|[Dd]aily|[Ss]tandup", regex=True) & (df.Time_Decimal > 1) & (df.Time_Decimal < 5)]

,Date,Duration,Project,Task_Category,Task_Cost,Task_Description,User,Position,Cost_per_hour,Time_Decimal
0,2017-09-01,01:17:00,Revenify,Development,26.41,Feedback on budgets index PR and Standup meeti...,UI13,Senior Back End Developer,20.58,1.28
56,2017-09-08,03:17:10,Revenify,Development,67.63,Standup meeting and working on transunion conn...,UI13,Senior Back End Developer,20.58,3.29
90,2017-09-13,01:03:53,Revenify,Development,21.91,"Raising PR for Transunion integration, feedbac...",UI13,Senior Back End Developer,20.58,1.06
113,2017-09-15,01:09:57,Revenify,Development,23.99,Encrypt sensitive information in the database ...,UI13,Senior Back End Developer,20.58,1.17
136,2017-09-18,03:53:36,Revenify,Development,80.13,Resolving PR conflicts and Standup meeting,UI13,Senior Back End Developer,20.58,3.89
...,...,...,...,...,...,...,...,...,...,...
24786,2019-09-18,01:50:00,LearnSage,Development,96.98,meeting in offices,UI04,Senior Front End Developer,52.90,1.83
24789,2019-09-18,03:18:00,CandidatePool,Development,198.15,XXX : Daily Standup + Formulating new design i...,UI03,Senior Back End Developer,60.05,3.30
24819,2019-09-19,02:30:00,LearnSage,Development,132.25,office meeting,UI04,Senior Front End Developer,52.90,2.50
24890,2019-09-23,03:00:00,LearnSage,Development,158.70,"office meeting, modal gallery - full screen wh...",UI04,Senior Front End Developer,52.90,3.00


In [96]:
stop_word = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now", 'xxx', 'x']

In [97]:
pd.Series(' '.join(df.Task_Description).lower().split()).value_counts()[:100]

xxx            6616
on             5801
to             4850
the            4658
and            4516
               ... 
prospect        251
information     250
status          249
landing         248
end             247
Length: 100, dtype: int64

In [98]:
usefull_words = [word for word in pd.Series(' '.join(df.Task_Description).lower().split()).value_counts()[:100].index if word not in stop_word]

In [99]:
usefull_words

['-',
 'working',
 'new',
 'testing',
 'code',
 'add',
 'staging',
 'review',
 'meeting',
 'automation',
 'standup',
 'sprint',
 'page',
 'production',
 'update',
 'fafsa',
 'weekly',
 'issue',
 'changes',
 'management',
 'planning',
 'test',
 'create',
 'deployment',
 'experience',
 'fix',
 'design',
 'discussion',
 'call',
 'daily',
 'fixing',
 'adding',
 'data',
 'deploy',
 'integration',
 'user',
 'funnel',
 'team',
 'qa',
 'server',
 'setup',
 'release',
 'tests',
 'implement',
 'endpoint',
 'feature',
 'project',
 'debts',
 'backend',
 'bug',
 'client',
 'frontend',
 'sf',
 'creating',
 'continue',
 'issues',
 'flow',
 'program',
 'crm',
 'devops',
 'tested',
 '+',
 'based',
 'env',
 'fixes',
 'report',
 'api',
 '&',
 'prospect',
 'information',
 'status',
 'landing',
 'end']